# Probando LlamaIndex para sitios web

In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [2]:
## Configuración para debugging

from llama_index.core.callbacks import (
    CallbackManager,
    LlamaDebugHandler,
    CBEventType,
)

llama_debug = LlamaDebugHandler(print_trace_on_end=True)
callback_manager = CallbackManager([llama_debug])

In [3]:
from llama_index.core import Settings

Settings.callback_manager = callback_manager

Settings.chunk_overlap = 450

In [4]:
from llama_index.readers.web import SimpleWebPageReader

documents = SimpleWebPageReader(html_to_text=True).load_data(
    ["https://hacemosescuela.cba.gov.ar/materiales-educativos/"]
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
Starting new HTTPS connection (1): hacemosescuela.cba.gov.ar:443
DEBUG:urllib3.connectionpool:https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None
https://hacemosescuela.cba.gov.ar:443 "GET /materiales-educativos/ HTTP/1.1" 200 None


In [5]:
len(documents)

1

In [6]:
documents[0]

Document(id_='https://hacemosescuela.cba.gov.ar/materiales-educativos/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)\n    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)\n    * [Otro

In [7]:
from llama_index.core.node_parser import (
    HierarchicalNodeParser,
    SentenceSplitter,
)
node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=[2500, 1800, 900])
nodes = node_parser.get_nodes_from_documents(documents)
len(nodes)

DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Aquel primer
festejo](https://hacemosescuela.cb...
> Adding chunk: Aquel primer
festejo](https://hacemosescuela.cb...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: La revolución de las medidas” es una
secuencia ...
> Adding chunk: La revolución de las medidas” es una
secuencia ...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
> Adding chunk: [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Conocer y describir nuestro espacio de trabajo”...
> Adding chunk: Conocer y describir nuestro espacio de trabajo”...
DEBUG:llama_index.core.node_parser.node_utils:> Adding chunk: Aquel primer
festejo](https://hacemose

17

In [8]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes

In [9]:
leaf_nodes = get_leaf_nodes(nodes)
len(leaf_nodes)

9

In [10]:
root_nodes = get_root_nodes(nodes)
len(root_nodes)

3

In [11]:
import os
from dotenv import load_dotenv

load_dotenv('.env', override=True)

GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")

In [12]:
# define storage context
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import StorageContext


docstore = SimpleDocumentStore()

# insert nodes into docstore
docstore.add_documents(nodes)

# define storage context (will include vector store by default too)
storage_context = StorageContext.from_defaults(docstore=docstore, )


In [13]:
# imports
from llama_index.embeddings.gemini import GeminiEmbedding

model_name = "models/embedding-001"

embed_model = GeminiEmbedding(
    model_name=model_name, api_key=GOOGLE_API_KEY, title="this is a document"
)

In [14]:
## Load index into vector index
from llama_index.core import VectorStoreIndex

base_index = VectorStoreIndex(
    leaf_nodes,
    storage_context=storage_context,
    embed_model=embed_model
)

**********
Trace: index_construction
    |_CBEventType.EMBEDDING -> 3.525057 seconds
**********


In [15]:
from llama_index.core.retrievers import AutoMergingRetriever

base_retriever = base_index.as_retriever(similarity_top_k=6)
retriever = AutoMergingRetriever(base_retriever, storage_context, verbose=True)

In [16]:
# query_str = "What were some lessons learned from red-teaming?"
# query_str = "Can you tell me about the key concepts for safety finetuning"
query_str = (
    "educación secundaria"
)

nodes = retriever.retrieve(query_str)

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 6341246f-da8f-4c9e-8d17-979150616839] [Similarity score:             0.7359] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 8415fd8f-8321-4cb0-b64f-544be7b2c894] [Similarity score:             0.725763] Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa....
> [Node 1d80ac65-2cbf-451c-ad82-ff04ff364c20] [Similarity score:             0.724245] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 5f91ae0c-c714-46a1-8ff1-6249778c0257] [Similarity score:             0.714791] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> [Node a8a05707-7c0b-4894-98e3-f083557556b0] [Similarity score:             0.709381] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](h...
> [Node 23

In [17]:
base_nodes = base_retriever.retrieve(query_str)

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 6341246f-da8f-4c9e-8d17-979150616839] [Similarity score:             0.7359] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 8415fd8f-8321-4cb0-b64f-544be7b2c894] [Similarity score:             0.725763] Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa....
> [Node 1d80ac65-2cbf-451c-ad82-ff04ff364c20] [Similarity score:             0.724245] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 5f91ae0c-c714-46a1-8ff1-6249778c0257] [Similarity score:             0.714791] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> [Node a8a05707-7c0b-4894-98e3-f083557556b0] [Similarity score:             0.709381] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](h...
> [Node 23



---



In [18]:
from llama_index.core.response.notebook_utils import display_source_node

for node in nodes:
    display_source_node(node, source_length=10000)

DEBUG:matplotlib:matplotlib data path: /home/illak/Proyectos/web_RAG_pruebas/.conda/lib/python3.11/site-packages/matplotlib/mpl-data
matplotlib data path: /home/illak/Proyectos/web_RAG_pruebas/.conda/lib/python3.11/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/illak/.config/matplotlib
CONFIGDIR=/home/illak/.config/matplotlib
DEBUG:matplotlib:interactive is False
interactive is False
DEBUG:matplotlib:platform is linux
platform is linux
DEBUG:matplotlib:CACHEDIR=/home/illak/.cache/matplotlib
CACHEDIR=/home/illak/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/illak/.cache/matplotlib/fontlist-v390.json
Using fontManager instance from /home/illak/.cache/matplotlib/fontlist-v390.json


**Node ID:** 8415fd8f-8321-4cb0-b64f-544be7b2c894<br>**Similarity:** 0.7257628669636027<br>**Text:** Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa...

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-iii/)

[Un recorrido por el jardín (Parte III)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-iii/)

“Un recorrido por el jardín (Parte III)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-ii/)

[Un recorrido por el jardín (Parte II)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-ii/)

“Un recorrido por el jardín (Parte II)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-i/)

[Un recorrido por el jardín (Parte I)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-i/)

“Un recorrido por el jardín (Parte I)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/)

[En la red, pero sin enredarme (Parte
II)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/)

"En la red, pero sin enredarme (Parte II)" es una secuencia didáctica de Tu
Escuela en Casa para las salas de...

[](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)

[En la red, pero sin enredarme (Parte
I)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)

"En la red, pero sin enredarme (Parte I)" es una secuencia didáctica de Tu
Escuela en Casa para las salas de...

[](https://hacemosescuela.cba.gov.ar/habia-una-vez-te-lo-cuento-otra-vez/)

[Había una vez… ¡te lo cuento otra
vez!](https://hacemosescuela.cba.gov.ar/habia-una-vez-te-lo-cuento-otra-vez/)

Había una vez… ¡te lo cuento otra vez! es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más materiales del nivel Inicial
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)

## Educación Primaria

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-
plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

[Recomendación didáctica.<br>

**Node ID:** ade2062e-9901-4fe2-8e32-c646d67faccb<br>**Similarity:** 0.7242450938752326<br>**Text:** La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-ii/)

[La revolución de los microorganismos (Parte
II)](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-ii/)

“La revolución de los microorganismos (Parte II)” es una secuencia didáctica
de Hacemos Escuela para 1.° y 2.° año del nivel...

[](https://hacemosescuela.cba.gov.ar/experiencias-escolares-que-desafian-el-
aula-estandar/)

[Experiencias escolares que desafían el aula
estándar](https://hacemosescuela.cba.gov.ar/experiencias-escolares-que-
desafian-el-aula-estandar/)

Objeto hipermedial interactivo (OHI) diseñado en el marco de la Actualización
Académica dirigida al Nivel de Educación Superior:...

[](https://hacemosescuela.cba.gov.ar/biblioteca-pedagogica/)

[Biblioteca pedagógica](https://hacemosescuela.cba.gov.ar/biblioteca-
pedagogica/)

Objeto hipermedial interactivo (OHI) que brinda una amplia selección de libros
y películas recopilados en una biblioteca...

[](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-i/)

[La revolución de los microorganismos (Parte
I)](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-i/)

“La revolución de los microorganismos (Parte I)” es una secuencia didáctica de
Hacemos Escuela para 1.° y 2.° año del nivel...

[ __ Ver más materiales del nivel Secundario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=165&key=)

![ISEP](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/isep-footer.png)

Los contenidos que se ponen a disposición en esta plataforma son creados y
curados por el Instituto Superior de Estudios Pedagógicos (ISEP), con el
aporte en la producción de los equipos técnicos de las diferentes Direcciones
Generales del Ministerio de Educación de la provincia de Córdoba.

![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/correo-footer.png)

###### **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/ig.png)

###### **¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ Licencia Creative Commons Atribución-
NoComercial 4.0 Internacional.](https://creativecommons.org/licenses/by-
nc/4.0/deed.es)**

![Licencias](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/licencias.png)

![Logo Footer](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_footer_2024.png)<br>

**Node ID:** 64d261b0-9a75-49f1-a8be-1f4437f4e0c4<br>**Similarity:** 0.717220301950153<br>**Text:** Aquel primer
festejo](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-aquel-primer-
festejo/)

25 de mayo: aquel primer festejo es una secuencia didáctica de Tu Escuela en
Casa destinada a las salas de 4 y 5 años de...

[](https://hacemosescuela.cba.gov.ar/25-de-mayo-otra-vuelta-por-la-historia/)

[25 de mayo: otra vuelta por la
historia](https://hacemosescuela.cba.gov.ar/25-de-mayo-otra-vuelta-por-la-
historia/)

25 de mayo: otra vuelta por la historia es una secuencia didáctica de Tu
Escuela en Casa destinada a 4.º, 5.° y 6.º grado...

[](https://hacemosescuela.cba.gov.ar/25-de-mayo-ninas-y-ninos-en-1810/)

[25 de Mayo: niñas y niños en 1810](https://hacemosescuela.cba.gov.ar/25-de-
mayo-ninas-y-ninos-en-1810/)

25 de Mayo: niñas y niños en 1810 es una secuencia didáctica de Tu Escuela en
Casa destinada a 2.º y 3.º grado de nivel Primario....

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-
pluriversos/)

[Recomendación didáctica: mundos
pluriversos](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-
pluriversos/)

“Mundos pluriversos” es una recomendación didáctica de Hacemos Escuela que
recupera diversos materiales del repositorio digital...

[](https://hacemosescuela.cba.gov.ar/unidades-de-longitud-el-metro-la-
revolucion-de-las-medidas/)

[Unidades de longitud: el metro. La revolución de las
medidas](https://hacemosescuela.cba.gov.ar/unidades-de-longitud-el-metro-la-
revolucion-de-las-medidas/)

“Unidades de longitud: el metro. La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-los-numeros-a-lo-largo-
de-la-escolaridad/)

[La enseñanza de los números a lo largo de la
escolaridad](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-los-numeros-a-
lo-largo-de-la-escolaridad/)

Objeto hipermedial interactivo para enseñar y aprender que brinda materiales
sobre contenidos vinculados con la enseñanza...

[](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-la-descripcion-a-lo-
largo-de-la-escolaridad/)

[La enseñanza de la descripción a lo largo de la
escolaridad](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-la-descripcion-
a-lo-largo-de-la-escolaridad/)

Objeto hipermedial interactivo para enseñar y aprender que ofrece materiales
sobre las prácticas del lenguaje vinculadas...

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-biodiversidad-en-
la-provincia-de-cordoba/)

[Recomendación didáctica: Biodiversidad en la Provincia de
Córdoba](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-
biodiversidad-en-la-provincia-de-cordoba/)

“Biodiversidad en la provincia de Córdoba” es una recomendación didáctica de
Hacemos Escuela destinada al Primer y Segundo...

[](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-asombrosos-
fotomontajes-ludicos/)

[Fotografía en el aula. Paseos asombrosos, fotomontajes
lúdicos](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-
asombrosos-fotomontajes-ludicos/)

“Fotografía en el aula. Paseos asombrosos, fotomontajes lúdicos” es una
secuencia didáctica de Hacemos Escuela destinada...

[](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-tu-mesa/)

[De la colmena a tu mesa](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-
tu-mesa/)

"De la colmena a tu mesa" es una secuencia didáctica de Tu Escuela en Casa
destinada a 1.º grado de nivel Primario....

[ __ Ver más materiales del nivel Primario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=161&key=)

## Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-
plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

[Recomendación didáctica. Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-
en-el-plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

“Enfoques en el plano: documentos y aportes para la enseñanza de Artes
Visuales en las escuelas” es una recomendación didáctica...

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,
reivindicaciones y luchas es una compilación de registros...

[](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-revolucion-
y-guerra/)

[Efemérides: 25 de Mayo. Patria, revolución y
guerra](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-
revolucion-y-guerra/)

Efemérides: 25 de Mayo.<br>

**Node ID:** 5f91ae0c-c714-46a1-8ff1-6249778c0257<br>**Similarity:** 0.7147914215776487<br>**Text:** [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversatorios)
  * Modalidades
    * [Jóvenes y Adultos](https://hacemosescuela.cba.gov.ar/jovenes-y-adultos)
    * [Educación Especial](https://hacemosescuela.cba.gov.ar/educacion-especial)
    * [Domiciliaria y Hospitalaria](https://hacemosescuela.cba.gov.ar/educacion-domiciliaria-y-hospitalaria)

![](https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-
materiales-educativos-1024x267.jpg)

##

## Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tienen el propósito de ofrecer desarrollos didácticos para
acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de
los docentes de todos los niveles.

Estos materiales se inscriben en los contenidos y aprendizajes priorizados por
los programas del Ministerio de Educación.

## Educación Inicial

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo.<br>

In [19]:
for node in base_nodes:
    display_source_node(node, source_length=10000)

**Node ID:** 6341246f-da8f-4c9e-8d17-979150616839<br>**Similarity:** 0.7359004959511858<br>**Text:** La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-los-numeros-a-lo-largo-
de-la-escolaridad/)

[La enseñanza de los números a lo largo de la
escolaridad](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-los-numeros-a-
lo-largo-de-la-escolaridad/)

Objeto hipermedial interactivo para enseñar y aprender que brinda materiales
sobre contenidos vinculados con la enseñanza...

[](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-la-descripcion-a-lo-
largo-de-la-escolaridad/)

[La enseñanza de la descripción a lo largo de la
escolaridad](https://hacemosescuela.cba.gov.ar/la-ensenanza-de-la-descripcion-
a-lo-largo-de-la-escolaridad/)

Objeto hipermedial interactivo para enseñar y aprender que ofrece materiales
sobre las prácticas del lenguaje vinculadas...

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-biodiversidad-en-
la-provincia-de-cordoba/)

[Recomendación didáctica: Biodiversidad en la Provincia de
Córdoba](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-
biodiversidad-en-la-provincia-de-cordoba/)

“Biodiversidad en la provincia de Córdoba” es una recomendación didáctica de
Hacemos Escuela destinada al Primer y Segundo...

[](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-asombrosos-
fotomontajes-ludicos/)

[Fotografía en el aula. Paseos asombrosos, fotomontajes
lúdicos](https://hacemosescuela.cba.gov.ar/fotografia-en-el-aula-paseos-
asombrosos-fotomontajes-ludicos/)

“Fotografía en el aula. Paseos asombrosos, fotomontajes lúdicos” es una
secuencia didáctica de Hacemos Escuela destinada...

[](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-tu-mesa/)

[De la colmena a tu mesa](https://hacemosescuela.cba.gov.ar/de-la-colmena-a-
tu-mesa/)

"De la colmena a tu mesa" es una secuencia didáctica de Tu Escuela en Casa
destinada a 1.º grado de nivel Primario....

[ __ Ver más materiales del nivel Primario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=161&key=)

## Educación Secundaria

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-
plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

[Recomendación didáctica.<br>

**Node ID:** 8415fd8f-8321-4cb0-b64f-544be7b2c894<br>**Similarity:** 0.7257628669636027<br>**Text:** Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa...

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-iii/)

[Un recorrido por el jardín (Parte III)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-iii/)

“Un recorrido por el jardín (Parte III)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-ii/)

[Un recorrido por el jardín (Parte II)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-ii/)

“Un recorrido por el jardín (Parte II)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/un-recorrido-por-el-jardin-parte-i/)

[Un recorrido por el jardín (Parte I)](https://hacemosescuela.cba.gov.ar/un-
recorrido-por-el-jardin-parte-i/)

“Un recorrido por el jardín (Parte I)” es una secuencia didáctica de Tu
Escuela en Casa para sala de 5 años del nivel Inicial....

[](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/)

[En la red, pero sin enredarme (Parte
II)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-ii/)

"En la red, pero sin enredarme (Parte II)" es una secuencia didáctica de Tu
Escuela en Casa para las salas de...

[](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)

[En la red, pero sin enredarme (Parte
I)](https://hacemosescuela.cba.gov.ar/en-la-red-pero-sin-enredarme-parte-i/)

"En la red, pero sin enredarme (Parte I)" es una secuencia didáctica de Tu
Escuela en Casa para las salas de...

[](https://hacemosescuela.cba.gov.ar/habia-una-vez-te-lo-cuento-otra-vez/)

[Había una vez… ¡te lo cuento otra
vez!](https://hacemosescuela.cba.gov.ar/habia-una-vez-te-lo-cuento-otra-vez/)

Había una vez… ¡te lo cuento otra vez! es una secuencia didáctica de Tu
Escuela en Casa destinada a las salas de 4...

[ __ Ver más materiales del nivel Inicial
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=158&key=)

## Educación Primaria

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-
plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

[Recomendación didáctica.<br>

**Node ID:** 1d80ac65-2cbf-451c-ad82-ff04ff364c20<br>**Similarity:** 0.7242450938752326<br>**Text:** La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-ii/)

[La revolución de los microorganismos (Parte
II)](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-ii/)

“La revolución de los microorganismos (Parte II)” es una secuencia didáctica
de Hacemos Escuela para 1.° y 2.° año del nivel...

[](https://hacemosescuela.cba.gov.ar/experiencias-escolares-que-desafian-el-
aula-estandar/)

[Experiencias escolares que desafían el aula
estándar](https://hacemosescuela.cba.gov.ar/experiencias-escolares-que-
desafian-el-aula-estandar/)

Objeto hipermedial interactivo (OHI) diseñado en el marco de la Actualización
Académica dirigida al Nivel de Educación Superior:...

[](https://hacemosescuela.cba.gov.ar/biblioteca-pedagogica/)

[Biblioteca pedagógica](https://hacemosescuela.cba.gov.ar/biblioteca-
pedagogica/)

Objeto hipermedial interactivo (OHI) que brinda una amplia selección de libros
y películas recopilados en una biblioteca...

[](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-i/)

[La revolución de los microorganismos (Parte
I)](https://hacemosescuela.cba.gov.ar/la-revolucion-de-los-microorganismos-
parte-i/)

“La revolución de los microorganismos (Parte I)” es una secuencia didáctica de
Hacemos Escuela para 1.° y 2.° año del nivel...

[ __ Ver más materiales del nivel Secundario
](http://hacemosescuela.cba.gov.ar/buscador/?nivel=165&key=)

![ISEP](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/isep-footer.png)

Los contenidos que se ponen a disposición en esta plataforma son creados y
curados por el Instituto Superior de Estudios Pedagógicos (ISEP), con el
aporte en la producción de los equipos técnicos de las diferentes Direcciones
Generales del Ministerio de Educación de la provincia de Córdoba.

![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/correo-footer.png)

###### **hacemosescuela@isep-cba.edu.ar**

[ ![](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/ig.png)

###### **¡Seguinos!**

](https://www.instagram.com/HacemosEscuela.Cba/)

**Este material está bajo una[ Licencia Creative Commons Atribución-
NoComercial 4.0 Internacional.](https://creativecommons.org/licenses/by-
nc/4.0/deed.es)**

![Licencias](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/licencias.png)

![Logo Footer](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_footer_2024.png)<br>

**Node ID:** 5f91ae0c-c714-46a1-8ff1-6249778c0257<br>**Similarity:** 0.7147914215776487<br>**Text:** [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacemosescuela.cba.gov.ar/conversatorios)
  * Modalidades
    * [Jóvenes y Adultos](https://hacemosescuela.cba.gov.ar/jovenes-y-adultos)
    * [Educación Especial](https://hacemosescuela.cba.gov.ar/educacion-especial)
    * [Domiciliaria y Hospitalaria](https://hacemosescuela.cba.gov.ar/educacion-domiciliaria-y-hospitalaria)

![](https://hacemosescuela.cba.gov.ar/wp-content/uploads/2022/11/Banner-
materiales-educativos-1024x267.jpg)

##

## Materiales educativos

En este espacio encontrarán materiales ordenados por nivel educativo. Fueron
diseñados para diferentes escenarios escolares: presencial, no presencial o
combinado. Tienen el propósito de ofrecer desarrollos didácticos para
acompañar la escolaridad y el diseño de propuestas de enseñanza por parte de
los docentes de todos los niveles.

Estos materiales se inscriben en los contenidos y aprendizajes priorizados por
los programas del Ministerio de Educación.

## Educación Inicial

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo.<br>

**Node ID:** a8a05707-7c0b-4894-98e3-f083557556b0<br>**Similarity:** 0.7093812976063829<br>**Text:** Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-
en-el-plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-
escuelas/)

“Enfoques en el plano: documentos y aportes para la enseñanza de Artes
Visuales en las escuelas” es una recomendación didáctica...

[](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-
los-trabajadores-derechos-reivindicaciones-y-luchas/)

[Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos,
reivindicaciones y
luchas](https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-
las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/)

Efemérides: 1.º de Mayo. El trabajo, las y los trabajadores: derechos,
reivindicaciones y luchas es una compilación de registros...

[](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-revolucion-
y-guerra/)

[Efemérides: 25 de Mayo. Patria, revolución y
guerra](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-
revolucion-y-guerra/)

Efemérides: 25 de Mayo.<br>

**Node ID:** 23cf13d3-4189-483e-87f4-5057feffc33e<br>**Similarity:** 0.7063791122928902<br>**Text:** Aquel primer
festejo](https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-aquel-primer-
festejo/)

25 de mayo: aquel primer festejo es una secuencia didáctica de Tu Escuela en
Casa destinada a las salas de 4 y 5 años de...

[](https://hacemosescuela.cba.gov.ar/25-de-mayo-otra-vuelta-por-la-historia/)

[25 de mayo: otra vuelta por la
historia](https://hacemosescuela.cba.gov.ar/25-de-mayo-otra-vuelta-por-la-
historia/)

25 de mayo: otra vuelta por la historia es una secuencia didáctica de Tu
Escuela en Casa destinada a 4.º, 5.° y 6.º grado...

[](https://hacemosescuela.cba.gov.ar/25-de-mayo-ninas-y-ninos-en-1810/)

[25 de Mayo: niñas y niños en 1810](https://hacemosescuela.cba.gov.ar/25-de-
mayo-ninas-y-ninos-en-1810/)

25 de Mayo: niñas y niños en 1810 es una secuencia didáctica de Tu Escuela en
Casa destinada a 2.º y 3.º grado de nivel Primario....

[](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-
pluriversos/)

[Recomendación didáctica: mundos
pluriversos](https://hacemosescuela.cba.gov.ar/recomendacion-didactica-mundos-
pluriversos/)

“Mundos pluriversos” es una recomendación didáctica de Hacemos Escuela que
recupera diversos materiales del repositorio digital...

[](https://hacemosescuela.cba.gov.ar/unidades-de-longitud-el-metro-la-
revolucion-de-las-medidas/)

[Unidades de longitud: el metro. La revolución de las
medidas](https://hacemosescuela.cba.gov.ar/unidades-de-longitud-el-metro-la-
revolucion-de-las-medidas/)

“Unidades de longitud: el metro.<br>

## Plug it into Query Engine


In [20]:
from llama_index.llms.gemini import Gemini

llm = Gemini()

In [21]:
from llama_index.core.query_engine import RetrieverQueryEngine

In [22]:
qa_prompt_str  = (
    "Contexto:\n"
    "---------------------\n"
    "{context_str}\n"
    "---------------------\n\n"
    "Usted es un ayudante y debe responder a la pregunta del usuario usando el contexto provisto.\n"
    "Responda  de forma amable, detallada y extensa.\n"
    "Además responda con enlaces a los recursos en caso de que sea posible.\n"
    """Si no encuentra la respuesta en el contexto responda: "No pude encontrar información en el contexto provisto".\n\n"""
    "Pregunta: {query_str}\n"
)

In [23]:
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# Text QA Prompt
chat_text_qa_msgs = [
    ChatMessage(
        role=MessageRole.SYSTEM,
        content=(
            "No responda si el contexto no contiene información suficiente."
        ),
    ),
    ChatMessage(role=MessageRole.USER, content=qa_prompt_str ),
]
text_qa_template = ChatPromptTemplate(chat_text_qa_msgs)

In [24]:
query_engine = RetrieverQueryEngine.from_args(retriever,
                                              llm=llm,
                                              text_qa_template=text_qa_template, 
                                              verbose=True, 
                                              callback_manager=callback_manager)
#base_query_engine = RetrieverQueryEngine.from_args(base_retriever)

In [25]:
query_str = (
    "qué material hay para la educación secundaria?"
)

In [26]:
response = query_engine.query(query_str)

DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 6341246f-da8f-4c9e-8d17-979150616839] [Similarity score:             0.735437] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 5f91ae0c-c714-46a1-8ff1-6249778c0257] [Similarity score:             0.719134] [ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403...
> [Node 1d80ac65-2cbf-451c-ad82-ff04ff364c20] [Similarity score:             0.712653] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 8415fd8f-8321-4cb0-b64f-544be7b2c894] [Similarity score:             0.712404] Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa....
> [Node a8a05707-7c0b-4894-98e3-f083557556b0] [Similarity score:             0.69454] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](h...
> [Node 8

In [27]:
print(str(response))

**Materiales para Educación Secundaria:**

* **Recomendación didáctica: Enfoques en el plano: documentos y aportes para la enseñanza de Artes Visuales en las escuelas:** https://hacemosescuela.cba.gov.ar/recomendacion-didactica-enfoques-en-el-plano-documentos-y-aportes-para-la-ensenanza-de-artes-visuales-en-las-escuelas/
* **Efemérides: 1.º de Mayo El trabajo, las y los trabajadores: derechos, reivindicaciones y luchas:** https://hacemosescuela.cba.gov.ar/efemerides-1-o-de-mayo-el-trabajo-las-y-los-trabajadores-derechos-reivindicaciones-y-luchas/
* **Efemérides: 25 de Mayo. Patria, revolución y guerra:** https://hacemosescuela.cba.gov.ar/efemerides-25-de-mayo-patria-revolucion-y-guerra/
* **25 de Mayo: protagonistas de una revolución:** https://hacemosescuela.cba.gov.ar/25-de-mayo-protagonistas-de-una-revolucion/
* **La revolución de las lenguas y los lectos:** https://hacemosescuela.cba.gov.ar/la-revolucion-de-las-lenguas-y-los-lectos-2/
* **La probabilidad. La revolución de los datos

## Probando Sub Questions

In [28]:
import nest_asyncio 
nest_asyncio.apply()

In [30]:
from llama_index.core.query_engine import SubQuestionQueryEngine
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# setup base query engine as tool
query_engine_tools = [
    QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name="sitios web",
            description="Sitios del Instituto Superior de Estudios Pedagógicos",
        ),
    ),
]

query_engine = SubQuestionQueryEngine.from_defaults(
    llm=llm,
    query_engine_tools=query_engine_tools,
    use_async=True,
)


In [31]:
response = query_engine.query(
    "qué material hay para la educación secundaria?"
)

Generated 1 sub questions.
[sitios web] Q: Sitios web del Instituto Superior de Estudios Pedagógicos
Generated 1 sub questions.
[sitios web] Q: Sitios web del Instituto Superior de Estudios Pedagógicos
DEBUG:llama_index.core.indices.utils:> Top 6 nodes:
> [Node 1d80ac65-2cbf-451c-ad82-ff04ff364c20] [Similarity score:             0.742761] La revolución de las medidas” es una
secuencia didáctica de Hacemos Escuela destinada al...

[](h...
> [Node 4aaf1a2a-dc10-4734-b7ce-c41fa46e9844] [Similarity score:             0.742271] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas](h...
> [Node 8415fd8f-8321-4cb0-b64f-544be7b2c894] [Similarity score:             0.741616] Conocer y describir nuestro espacio de trabajo” es una
secuencia didáctica de Tu Escuela en Casa....
> [Node a8a05707-7c0b-4894-98e3-f083557556b0] [Similarity score:             0.736671] Enfoques en el plano: documentos y aportes para la
enseñanza de Artes Visuales en las
escuelas]

In [33]:
print(str(response))

The provided context does not mention anything about materials for secondary education, so I cannot answer this question from the provided context.


## Debugging

In [61]:
llama_debug.get_event_pairs()

[[CBEvent(event_type=<CBEventType.NODE_PARSING: 'node_parsing'>, payload={<EventPayload.DOCUMENTS: 'documents'>: [Document(id_='https://hacemosescuela.cba.gov.ar/materiales-educativos/', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](https://hacemosescuela.

In [62]:
llama_debug.get_llm_inputs_outputs()

[[CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\n[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas]

In [63]:
event_pairs = llama_debug.get_llm_inputs_outputs()

print(event_pairs[0][0])
print(event_pairs[0][1].payload.keys())
print(event_pairs[0][1].payload["response"])

CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\n[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](h

In [64]:
print(event_pairs[0][0])
print(event_pairs[0][0].payload.keys())


CBEvent(event_type=<CBEventType.LLM: 'llm'>, payload={<EventPayload.MESSAGES: 'messages'>: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='No responda si el contexto no contiene información suficiente.', additional_kwargs={}), ChatMessage(role=<MessageRole.USER: 'user'>, content='Contexto:\n---------------------\n[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-\ncontent/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)\n](https://hacemosescuela.cba.gov.ar) __\n\n  * [Inicio](https://hacemosescuela.cba.gov.ar)\n  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)\n  * Materiales educativos\n    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)\n    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)\n    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)\n    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)\n  * Programas\n    * [Entre Lenguas](h

In [65]:
print(event_pairs[0][0].payload["messages"][1])

user: Contexto:
---------------------
[ ![Logo](https://hacemosescuela.cba.gov.ar/wp-
content/themes/hacemosescuela_2-8/img/logo_202403.png?v=20240411-1)
](https://hacemosescuela.cba.gov.ar) __

  * [Inicio](https://hacemosescuela.cba.gov.ar)
  * [Acerca de](https://hacemosescuela.cba.gov.ar/acerca-de)
  * Materiales educativos
    * [Secuencias didácticas](https://hacemosescuela.cba.gov.ar/materiales-educativos)
    * [Objetos Hipermediales Interactivos](https://hacemosescuela.cba.gov.ar/ohi)
    * [Colecciones didácticas](https://hacemosescuela.cba.gov.ar/colecciones-didacticas)
    * [Recomendaciones](https://hacemosescuela.cba.gov.ar/recomendaciones)
  * Programas
    * [Entre Lenguas](https://hacemosescuela.cba.gov.ar/entre-lenguas)
    * [Cultura Digital](https://hacemosescuela.cba.gov.ar/cultura-digital)
    * [Otros programas](https://hacemosescuela.cba.gov.ar/programas)
  * [Comunidad](https://hacemosescuela.cba.gov.ar/comunidad-de-practicas)
  * [Conversatorios](https://hacem

In [66]:
# Clear the currently cached events
llama_debug.flush_event_logs()